In [1]:
# install dependencies and make imports
!pip install plotly; 
!pip install dash;
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
app = dash.Dash(__name__)

In [41]:
# read data for map and perform df ops
income = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/map/income_per_state.csv")
obesity = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/map/obesity_per_state.csv")
population = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/map/population_per_state.csv")
number_bk = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/locations/burgerking_locations.csv")
number_chick = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/locations/chick-fil-a_locations.csv")
number_mcdonalds = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/locations/mcdonalds_locations.csv")
number_starbucks = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/locations/starbucks_locations.csv")
number_subway = pd.read_csv("/Users/srijeevsarkar/Desktop/IAT/Datasets/locations/subway_locations.csv")

# merging datasets
df = pd.merge(income, obesity, on=['State'])
master = pd.merge(df, population, on=['State'])
rest1 = pd.merge(number_bk,number_chick, on=['State'])
rest2 = pd.merge(rest1,number_mcdonalds, on=['State'])
rest3 = pd.merge(rest2,number_starbucks, on=['State'])
rest4 = pd.merge(rest3,number_subway, on=['State'])
rest = rest4

# dictionary to add state codes to the dataframes
master.rename(columns={'Average Income (in K)': 'Income', 'Percent of obesities': 'Obesity', 'Population (July)': 'Population'}, inplace=True)
state_2 = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

state_2 = {state: abbrev for abbrev, state in state_2.items()}

df_new = pd.DataFrame(list(state_2.items()),columns = ['State Code','State'])

final = pd.merge(master,df_new,on=['State'])

rest = pd.merge(rest,df_new,on=['State'])

In [43]:
# view dataframe (skip)
# final.head(10)
#rest.dtypes
rest.head(10)

,State,Number of Buger King,Number of Chick-fil-A,Number of McDonalds,Number of Starbucks,Number of Subway,State Code
0,Alabama,8.0,66.0,210.0,85.0,418.0,AL
1,Alaska,164.0,0.0,27.0,49.0,62.0,AK
2,Arizona,163.0,23.0,212.0,488.0,412.0,AZ
3,Arkansas,127.0,26.0,127.0,55.0,270.0,AR
4,California,583.0,59.0,1165.0,2821.0,2353.0,CA
5,Colorado,123.0,29.0,181.0,481.0,394.0,CO
6,Connecticut,65.0,0.0,143.0,123.0,306.0,CT
7,Delaware,20.0,6.0,34.0,25.0,27.0,DE
8,Florida,561.0,164.0,728.0,694.0,1349.0,FL
9,Georgia,260.0,212.0,365.0,326.0,788.0,GA


In [55]:
# Chloropleth

dff = final.copy()
option_slctd = "Income"

fig = px.choropleth(
    data_frame=dff,
    locationmode='USA-states',
    locations='State Code',
    scope="usa",
    color=option_slctd,
    color_continuous_scale="Viridis",
    hover_data=['State', option_slctd],
    template='plotly_dark'
)

# Scatter

dff2 = rest.copy()
option_slctd_2 = "Number of McDonalds"

fig2 = px.scatter_geo(
     data_frame=dff2,
     locationmode='USA-states',
     scope="usa",
     locations="State Code",
     size=option_slctd_2,
     template='plotly_dark',
                   )

fig.add_trace(
    fig2.data[0]
)
#fig.show();

In [53]:
# App layout 
app.layout = html.Div([

    html.H1("Fast Food Analysis Dashboard", style={'text-align': 'center'}),

    dcc.Dropdown(id="select_category",
                 options=[
                     {"label": "Income per state", "value": 'Income'},
                     {"label": "Obesity per state", "value": 'Obesity'},
                     {"label": "Population per state", "value": 'Population'}],
                 multi=False,
                 value="Income",
                 style={'width': "40%"}
                 ),
     
    dcc.Dropdown(id="select_restaurant",
                 options=[
                     {"label": "McDonalds", "value": 'Number of McDonalds'},
                     {"label": "Burger King", "value": 'Number of Buger King'},
                     {"label": "Subway", "value": 'Number of Subway'},
                     {"label": "Starbucks", "value": 'Number of Starbucks'},
                     {"label": "Chick-fil-A", "value": 'Number of Chick-fil-A'}],
                 multi=False,
                 value="Number of McDonalds",
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),
    dcc.Graph(id='fast_food_map', figure={})

])

In [58]:
# Connect the Plotly graphs with Dash Components
# The callback uses inputs and outputs for interactivity in the dashboard
# For x number of outputs the function defined for callback needs to return x number of items

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='fast_food_map', component_property='figure')],
    [Input(component_id='select_category', component_property='value'),
     Input(component_id='select_restaurant',component_property='value')]
)
def update_graph(option_slctd,option_slctd_2):
    
    container = "The option chosen by user was: {}".format(option_slctd)
    
    dff = final.copy()

    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='State Code',
        scope="usa",
        color=option_slctd,
        color_continuous_scale="Viridis",
        hover_data=['State', option_slctd],
        template='plotly_dark'
    )

    dff2 = rest.copy()

    fig2 = px.scatter_geo(
         data_frame=dff2,
         locationmode='USA-states',
         scope="usa",
         locations="State Code",
         size=option_slctd_2,
         template='plotly_dark',
                       )

    fig.add_trace(
        fig2.data[0]
    )

    return container,fig

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2021 18:51:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:51:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:52:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:52:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:52:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:52:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2021 18:54:33] "POST /_dash-update-component

In [ ]:
# Kill process
# !ps -a
# kill -9 (processID)